# Libraries

In [1]:
import numpy as np
import pandas as pd
import pystan
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

In [2]:
my_data = np.loadtxt('../Catalogue/Match07_smalldoubleclean_emlines.csv', delimiter=',', dtype=str)

In [3]:
my_dictionary = {}
for i in range(len(my_data[0, :])):                                         # Converting numpy array into dictionary
    my_dictionary[my_data[0, i]] = np.array(my_data[0 + 1:, i], dtype=str)

In [4]:
cataid       = my_dictionary['CATAID'].astype(str)
redshift     = my_dictionary['Z'].astype(float)
stellar_mass = my_dictionary['logmstar'].astype(float)   # stellar mass from sed fitting - log scale
survey       = my_dictionary['SURVEY'].astype(str)
uv_class     = my_dictionary['UV_CLASS_YI2011'].astype(str)
bpt_class    = my_dictionary['BPT_CLASS'].astype(str)
whan_class   = my_dictionary['WHAN_CLASS'].astype(str)

In [5]:
logit_class = []
indexes_of_interest = []
for i in range(cataid.size):
    if uv_class[i]=='UV_UPTURN':
        logit_class.append(1)
        indexes_of_interest.append(i)
    elif uv_class[i] == 'UV_WEAK':
        logit_class.append(0)
        indexes_of_interest.append(i)
    else:
        continue
logit_class = np.array(logit_class)

In [6]:
whan_class_numbers = []

for i in range(cataid.size):
    if whan_class[i]=='NA':
        whan_class_numbers.append(0)
    elif whan_class[i]=='Retired/Passive':
        whan_class_numbers.append(1)
    elif whan_class[i]=='wAGN':
        whan_class_numbers.append(2)
    elif whan_class[i]=='sAGN':
        whan_class_numbers.append(3)
    elif whan_class[i]=='SF':
        whan_class_numbers.append(4)
    else:
        print "Something is wrong"
whan_class_numbers = np.array(whan_class_numbers)

In [7]:
logit_dataframe                                   = {}
logit_dataframe['CATAID']                         = cataid[indexes_of_interest]
logit_dataframe['LOGIT_CLASS(1-UVUP;0-UVWEAK)']   = logit_class
logit_dataframe['Z']                              = redshift[indexes_of_interest]
logit_dataframe['STELLAR_MASS']                   = stellar_mass[indexes_of_interest]
logit_dataframe['UV_CLASS']                       = uv_class[indexes_of_interest]
logit_dataframe['BPT_CLASS']                      = bpt_class[indexes_of_interest]
logit_dataframe['WHAN_CLASS']                     = whan_class[indexes_of_interest]
logit_dataframe['WHAN(0-NA;1-RP;2-wA;3-sA;4-SF)'] = whan_class_numbers[indexes_of_interest]

In [8]:
logit_df = pd.DataFrame(logit_dataframe)

In [9]:
logit_df = logit_df.set_index('CATAID')

In [10]:
logit_df

,BPT_CLASS,LOGIT_CLASS(1-UVUP;0-UVWEAK),STELLAR_MASS,UV_CLASS,WHAN(0-NA;1-RP;2-wA;3-sA;4-SF),WHAN_CLASS,Z
CATAID,,,,,,,
593695,SF,0,10.174240,UV_WEAK,4,SF,0.13198
594913,NA,0,10.792920,UV_WEAK,1,Retired/Passive,0.13870
594994,NA,1,10.864960,UV_UPTURN,1,Retired/Passive,0.17168
598961,NA,1,10.762900,UV_UPTURN,0,NA,0.19304
600232,LINER,1,10.471420,UV_UPTURN,1,Retired/Passive,0.15761
601015,NA,0,10.738460,UV_WEAK,1,Retired/Passive,0.16470
609264,NA,1,11.092890,UV_UPTURN,2,wAGN,0.21197
609352,NA,1,11.106870,UV_UPTURN,4,SF,0.30252
609836,LINER,1,10.666410,UV_UPTURN,1,Retired/Passive,0.13537


In [11]:
logit_df.to_csv('./../Catalogue/binom_reg_dataset.csv', index=True)